<a href="https://colab.research.google.com/github/cbologa/single-cell/blob/main/chondrocyte.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Check RAM available (256 GB recommended)

In [ ]:
install.packages("memuse")
invisible(gc())
memuse::Sys.meminfo()

Install required packages

In [ ]:
install.packages(c("Matrix", 
                   "data.table", 
                   "caret", 
                   "doParallel", 
                   "xgboost", 
                   "Rmisc", 
                   "matrixStats",
                   "openxlsx"))

Load packages

In [ ]:
library(Matrix)
library(data.table)
library(caret)
#library(doParallel)
library(xgboost)
library(Rmisc)
library(matrixStats)
library(openxlsx)

Read dataset files

In [ ]:
DATASET <- "E-CURD-112"

counts <- readMM(paste0(DATASET,".aggregated_filtered_normalised_counts.mtx"))
rows <- fread(paste0(DATASET,".aggregated_filtered_normalised_counts.mtx_rows"), header = F)
cols <- fread(paste0(DATASET,".aggregated_filtered_normalised_counts.mtx_cols"), header = F)
labels <- fread(paste0("ExpDesign-",DATASET,".tsv"))
invisible(gc())

Merge files

In [ ]:
rownames(counts) <- rows$V2
colnames(counts) <- cols$V1

Convert to data.table

In [ ]:
dt <- as.data.table(as.matrix(t(counts)), keep.rownames = T)
invisible(gc())

Add cell labels

In [ ]:
labels <- labels[,c("Assay","Factor Value[inferred cell type - ontology labels]")]
names(labels) <- c("rn","CellType")
labels[,Y:=ifelse(CellType=="chondrocyte", "active", "inactive")]
labels[,Y:=factor(Y)]
dt <- merge(labels, dt, by="rn")

Split dataset into labeled/unlabeled cells

In [ ]:
dtl <- dt[CellType!=""]
dtu <- dt[CellType==""]
saveRDS(dtu, "dtu.rds")
saveRDS(dtl, "dtl.rds")

Clear memory

In [ ]:
rm(DATASET,counts,rows,cols,labels,dt,dtu); invisible(gc())

Split labeled data into 5 groups

In [ ]:
set.seed(1234)
folds <- createFolds(dtl$Y,k=5)

Add empty column to store prediction results

In [ ]:
dtl1 <- data.table(dtl[,.(rn,CellType,Y)],
                   Y_PRED=character(nrow(dtl)),
                   Y_PROB=numeric(nrow(dtl)))

Create empty list to store the 5 XGBoost models

In [ ]:
model_list <- list()

For each group build a model using the rest of the data and make predictions

In [ ]:
for(i in 1:length(folds)){
  print(i)
  #i <- 1
  fold <- folds[[i]]
  print("folds")
  
  test.set <- dtl[fold]
  train.set <- dtl[!fold]
  #print("train set")
  
  train_Y <- train.set[,.(Y=Y)]
  train_X <- train.set[,-c(1:3)]
  rm(train.set); invisible(gc())
  #print("trainX")
  # try to identify descriptors with low variance 
  train_nzv <- nearZeroVar(train_X, saveMetrics= TRUE)
  #print("train_nzv")
  clean1 <- train_X[,-train_nzv$nzv,with=F]
  rm(train_X);invisible(gc())
  #print("clean1")
  train_dt <- data.table(train_Y,clean1)
  rm(clean1);invisible(gc())
  #print("train_dt")
  
  # set the internal resampling method to simple 7-fold crossvalidation
  fitControl <- trainControl(method="cv", number=5, #allowParallel = T,
                             classProbs = T, summaryFunction = twoClassSummary)
  train_dt <- na.omit(train_dt)
  invisible(gc())
  #print("na_omit")
  rm(train_Y,train_nzv); invisible(gc())
  # use parallel processing for faster execution (when hardware resources permit)
  # not to be used when running on Google Colab
  cl <- makeCluster(16)      # 16 cores for 256 GB RAM, 8 cores with 128 GB RAM
  registerDoParallel(cl)     # register the number of cores for parallel execution
  
  print(system.time({
    m <- train(Y ~ ., 
               data=train_dt, 
               method="xgbTree", 
               trControl=fitControl,
               metric="ROC",
               maximize=T,
               preProc = c("center", "scale"),
               scale_pos_weight=train_dt[Y=="inactive",.N]/train_dt[Y=="active",.N]
    )
    dtl1[fold,Y_PRED:=predict(m,test.set)]
    dtl1[fold,Y_PROB:=predict(m,test.set,type = "prob")[,1]]
    model_list[[i]] <- m
  }))  
  # shut down the parallel cluster
  stopCluster(cl)
  rm(m,test.set,train_dt); invisible(gc())
}


Save the 5 models and predictions for labeled cells

In [ ]:
saveRDS(model_list,"models5.rds")
fwrite(dtl1[,.(rn,CellType,Y,Y_PRED,Y_PROB)],"predicted.csv")

Display confusion table

In [ ]:
table(dtl1$Y,dtl1$Y_PRED)

Compute median probabilities by cell type to find which cells are most similar to chondrocytes

In [ ]:
dtl1[,pPROB:=-log10(Y_PROB)]
dt_type <- dtl1[,.(median_pPROB=median(pPROB)),by="CellType"]
setorder(dt_type,median_pPROB)
fwrite(dt_type,"cell_type_similarity_to_chondrocytes.csv")

Predict unlabeled cells

In [ ]:
dtu <- readRDS("dtu.rds")
dtu1 <-dtu[,.(rn,CellType,Y)]
for(i in 1:5){
  print(i)
  m <- model_list[[i]]
  dtu1[,c(paste0("pred",i)):=predict(m,dtu,type = "prob")[,1]]
}
dtu1[,Y_PROB:=rowMedians(as.matrix(.SD)),
     .SDcols = c('pred1', 'pred2', 'pred3', 'pred4', 'pred5')]
dtu1[,Y_PRED:=ifelse(Y_PROB>0.5,"active","inactive")]
fwrite(dtu1[,.(rn,CellType,Y,Y_PRED,Y_PROB)],"unlabeled_predicted.csv")

Feature importance

In [ ]:
dt_list <- list()
model_list <- readRDS("models5.rds")
for (i in 1:5){
  m <- model_list[[i]]
  df <- varImp(m)$importance
  df$gene <- row.names(df)
  setDT(df)
  df[,model:=paste0("m",i)]
  dt_list[[i]] <- df
}

dt_long <- rbindlist(dt_list)
dt_wide <- dcast(dt_long, gene~model, value.var = 'Overall')
dt_wide[,imp:= rowMeans(as.matrix(.SD),na.rm = T),
         .SDcols = c('m1', 'm2', 'm3', 'm4', 'm5')]
setorder(dt_wide,-imp)
fwrite(dt_wide[,.(gene,imp)],"varImp.csv")